In [1]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

import os
import itertools

import regex as re

import math

from IPython.display import HTML, display
import tabulate

In [2]:
nlp = Polish()    #spacy.load('pl_core_news_sm')
tokenizer = Tokenizer(nlp.vocab)

In [3]:
bill_filenames = []
for file in os.listdir("../ustawy"):
    if file.endswith(".txt"):
        bill_filenames.append(os.path.join("../ustawy", file))

In [4]:
freq = dict()
bigrams = dict()

In [5]:
for bill_name in bill_filenames:
    with open(bill_name, encoding='utf-8') as b_file:
        doc = b_file.read()
        doc = doc.lower()
        tokens = tokenizer(doc)
        
        prev = tokens[0].text
        freq[prev] = freq.get(prev, 0) + 1
        for i in range(1,len(tokens)):
            token = tokens[i].text
            bigram = prev + ' ' + token
            bigrams[bigram] = bigrams.get(bigram, 0) + 1
            prev = token
            freq[token] = freq.get(token, 0) + 1

In [6]:
bigrams = {k: v for k, v in sorted(bigrams.items(), key=lambda item: item[1], reverse=True)}
freq = {k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}

In [7]:
bigrams = dict(filter(lambda kv: re.match('^\p{L}+ \p{L}+$', kv[0]) is not None, bigrams.items()))
freq = dict(filter(lambda kv: re.match('^\p{L}+$', kv[0]) is not None, freq.items()))

## PMI

In [8]:
bigrams_PMI = {}
n_tokens = sum(freq.values())
n_bigrams = sum(bigrams.values())

In [9]:
for bigram in bigrams.keys():
    w1, w2 = bigram.split(' ')
    w1_p = freq[w1]/n_tokens
    w2_p = freq[w2]/n_tokens
    w1_w2_p = bigrams[bigram]/n_bigrams
    bigrams_PMI[bigram] = math.log2(w1_w2_p / (w1_p * w2_p))

In [10]:
bigrams_PMI = {k: v for k, v in sorted(bigrams_PMI.items(), key=lambda item: item[1], reverse=True)}

### TOP 10

In [11]:
for bigram, pmi in list(bigrams_PMI.items())[:10]:
    print(bigram + ': ' + str(pmi))

doktorów habilitowanych: 21.97577769832482
pionową ścianę: 21.97577769832482
punktem wyprowadzonym: 21.97577769832482
skrzynek lęgowych: 21.97577769832482
usprawnianie zaburzonych: 21.97577769832482
oświatowa nieobejmująca: 21.97577769832482
stępkę położono: 21.97577769832482
frachtem dystansowym: 21.97577769832482
wybuchła wojna: 21.97577769832482
dało pożytecznego: 21.97577769832482


In [12]:
bigrams_PMI_m5 = dict()

for bigram, pmi in bigrams_PMI.items():
    if bigrams[bigram] >= 5:
        bigrams_PMI_m5[bigram] = pmi
    

### TOP 10 with >= 5 occurencies

In [13]:
for bigram, pmi in list(bigrams_PMI_m5.items())[:10]:
    print(bigram + ': ' + str(pmi))

ręcznego miotacza: 19.653849603437457
świeckie przygotowujące: 19.653849603437457
grzegorz schetyna: 19.653849603437457
młyny kulowe: 19.653849603437457
zaszkodzić wynikom: 19.653849603437457
adama mickiewicza: 19.390815197603665
przeponowe rurowe: 19.390815197603665
mleczka makowego: 19.390815197603665
schedę spadkową: 19.168422776267217
lambrekiny okienne: 19.168422776267217


## LLR

In [14]:
bigrams_LLR = {}
all_occ = sum(bigrams.values())

In [15]:
c_w_cache = {}

def count_witout(word, without):
    if (word, without) in c_w_cache.keys():
        return c_w_cache[(word, without)]
    else:
        count = 0
        for bigram, freq in bigrams.items():
            if word in bigram and without not in bigram:
                    count += freq
        c_w_cache[(word, without)] = count
        return count

In [16]:
def H(args, N):
    res = 0
    for k in args:
        k /= N
        k_eq_0 = 1 if k == 0 else 0
        res += k * math.log(k + k_eq_0)
    return res

In [17]:
for bigram, k11 in bigrams.items():
    w1, w2 = bigram.split(' ')
    k12 = freq[w2] - k11
    k21 = freq[w1] - k11
    
    k22 = all_occ - (k11 + k12 + k21)
    LLR = 2 * all_occ * (H([k11, k12, k21, k22], all_occ) - H([k11 + k12, k21 + k22], all_occ) - H([k11 + k21, k12 + k22], all_occ))
    
    bigrams_LLR[bigram] = LLR   

In [18]:
bigrams_LLR = {k: v for k, v in sorted(bigrams_LLR.items(), key=lambda item: item[1], reverse=True)}

### TOP 10 LLR

In [19]:
for bigram, llr in list(bigrams_LLR.items())[:10]:
    print(bigram + ': ' + str(pmi))

mowa w: 19.168422776267217
których mowa: 19.168422776267217
o których: 19.168422776267217
którym mowa: 19.168422776267217
dodaje się: 19.168422776267217
do spraw: 19.168422776267217
o którym: 19.168422776267217
w w: 19.168422776267217
stosuje się: 19.168422776267217
minister właściwy: 19.168422776267217


## TRIGRAMS

In [20]:
trigrams = {}

In [21]:
for bill_name in bill_filenames:
    with open(bill_name, encoding='utf-8') as b_file:
        doc = b_file.read()
        doc = doc.lower()
        tokens = tokenizer(doc)
        
        prev2 = tokens[0].text
        prev1 = tokens[1].text
        for i in range(2,len(tokens)):
            token = tokens[i].text
            trigram = prev2 + ' ' + prev1 + ' ' + token
            trigrams[trigram] = trigrams.get(trigram, 0) + 1
            prev2 = prev1
            prev1 = token


In [22]:
trigrams = dict(filter(lambda kv: re.match('^\p{L}+ \p{L}+ \p{L}+$', kv[0]) is not None, trigrams.items()))
trigrams_PMI = {}
n_trigrams = sum(trigrams.values())

In [23]:
for trigram in trigrams.keys():
    w1, w2, w3 = trigram.split(' ')
    w1_p = freq[w1]/n_tokens
    w2_p = freq[w2]/n_tokens
    w3_p = freq[w2]/n_tokens

    w1_w2_w3_p = trigrams[trigram]/n_trigrams
    trigrams_PMI[trigram] = math.log(w1_w2_w3_p / (w1_p * w2_p * w3_p))

In [24]:
trigrams_PMI = {k: v for k, v in sorted(trigrams_PMI.items(), key=lambda item: item[1], reverse=True)}

### TOP 10

In [25]:
for trigram, pmi in list(trigrams_PMI.items())[:10]:
    print(trigram + ': ' + str(pmi))

doktorów habilitowanych nauk: 30.448705803472787
skrzynek lęgowych dla: 30.448705803472787
oświatowa nieobejmująca prowadzenia: 30.448705803472787
stępkę położono lub: 30.448705803472787
frachtem dystansowym jest: 30.448705803472787
wybuchła wojna grożąca: 30.448705803472787
poświęcenie objęło cały: 30.448705803472787
uprzywilejowanym wierzytelnościom z: 30.448705803472787
błędem nautycznym ubezpieczającego: 30.448705803472787
koszy ulicznych na: 30.448705803472787


In [26]:
trigrams_PMI_m5 = dict()

for trigram, pmi in trigrams_PMI.items():
    if trigrams[trigram] >= 5:
        trigrams_PMI_m5[trigram] = pmi

### TOP 10 with >= 5 occurencies

In [27]:
for trigram, pmi in list(trigrams_PMI_m5.items())[:10]:
    print(trigram + ': ' + str(pmi))

świeckie przygotowujące się: 27.229829978604588
zaszkodzić wynikom podjętych: 27.229829978604588
adama mickiewicza w: 26.86518686501668
lambrekiny okienne i: 26.71103618518942
chrześcijan baptystów w: 26.682865308222723
czyniąc uciążliwym korzystanie: 26.64204331370247
chwytów obezwładniających oraz: 26.556885505362164
jana matejki w: 26.536682798044644
jonów chlorków i: 26.28982272011312
uroczystości pogrzebowe i: 26.259051061446364


## LLR

In [28]:
trigrams_LLR = {}

In [29]:
for trigram, k11 in trigrams.items():
    #t1t2, t3
    w1, w2, w3 = trigram.split(' ')
    k12 = freq[w3] - k11
    k21 = bigrams[w1 + ' ' + w2] - k11
    
    k22 = all_occ - (k11 + k12 + k21)
    LLR = 2 * all_occ * (H([k11, k12, k21, k22], all_occ) - H([k11 + k12, k21 + k22], all_occ) - H([k11 + k21, k12 + k22], all_occ))
    
    trigrams_LLR[trigram] = LLR   

In [30]:
trigrams_LLR = {k: v for k, v in sorted(trigrams_LLR.items(), key=lambda item: item[1], reverse=True)}

### TOP 10 LLR

In [31]:
for trigram, llr in list(trigrams_LLR.items())[:10]:
    print(trigram + ': ' + str(pmi))

o których mowa: 26.259051061446364
o którym mowa: 26.259051061446364
których mowa w: 26.259051061446364
właściwy do spraw: 26.259051061446364
o której mowa: 26.259051061446364
którym mowa w: 26.259051061446364
ustawie z dnia: 26.259051061446364
minister właściwy do: 26.259051061446364
zastępuje się wyrazami: 26.259051061446364
wejścia w życie: 26.259051061446364


# PODSUMOWANIE

In [32]:
b_PMI_list = list(bigrams_PMI.keys())[:10]

b_PMI_list_5oc = list(bigrams_PMI_m5.keys())[:10]

b_LLR_list = list(bigrams_LLR.keys())[:10]


b_table = []

for i in range(10):
    b_table.append([b_PMI_list[i], b_PMI_list_5oc[i], b_LLR_list[i]])
    
t_PMI_list = list(trigrams_PMI.keys())[:10]

t_PMI_list_5oc = list(trigrams_PMI_m5.keys())[:10]

t_LLR_list = list(trigrams_LLR.keys())[:10]


t_table = []

for i in range(10):
    t_table.append([t_PMI_list[i], t_PMI_list_5oc[i], t_LLR_list[i]])


display(HTML('<h1>Bigrams</h1>'))
display(HTML(tabulate.tabulate(b_table, tablefmt='html', headers=['PMI - all', 'PMI - >=5occ', 'LLR'])))
display(HTML('<h1>Trigrams</h1>'))
display(HTML(tabulate.tabulate(t_table, tablefmt='html', headers=['PMI - all', 'PMI - >=5occ', 'LLR'])))

PMI - all,PMI - >=5occ,LLR
doktorów habilitowanych,ręcznego miotacza,mowa w
pionową ścianę,świeckie przygotowujące,których mowa
punktem wyprowadzonym,grzegorz schetyna,o których
skrzynek lęgowych,młyny kulowe,którym mowa
usprawnianie zaburzonych,zaszkodzić wynikom,dodaje się
oświatowa nieobejmująca,adama mickiewicza,do spraw
stępkę położono,przeponowe rurowe,o którym
frachtem dystansowym,mleczka makowego,w w
wybuchła wojna,schedę spadkową,stosuje się
dało pożytecznego,lambrekiny okienne,minister właściwy


PMI - all,PMI - >=5occ,LLR
doktorów habilitowanych nauk,świeckie przygotowujące się,o których mowa
skrzynek lęgowych dla,zaszkodzić wynikom podjętych,o którym mowa
oświatowa nieobejmująca prowadzenia,adama mickiewicza w,których mowa w
stępkę położono lub,lambrekiny okienne i,właściwy do spraw
frachtem dystansowym jest,chrześcijan baptystów w,o której mowa
wybuchła wojna grożąca,czyniąc uciążliwym korzystanie,którym mowa w
poświęcenie objęło cały,chwytów obezwładniających oraz,ustawie z dnia
uprzywilejowanym wierzytelnościom z,jana matejki w,minister właściwy do
błędem nautycznym ubezpieczającego,jonów chlorków i,zastępuje się wyrazami
koszy ulicznych na,uroczystości pogrzebowe i,wejścia w życie


### Answer the following questions:
#### Why do we have to filter the bigrams, rather than the token sequence?
Musimy filtrować bigramy, a nie tokeny gdyż mogłoby to spowodować utworzenie niepoprawnych bigramów, np: gdy weźmiemy pod uwagę zdanie: "drużyna piłkarska skłąda się z 11 zawodników", to przy filtrowaniu tokenów powstał by bigram z zawodników, który powinien zostać wyfiltrowany jako bigramy "z 11" i "11 zawodników".
#### Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
Według mnie dla bigramów lepiej działa PMI z filtrem, ponieważ znajduje ono chociażby imiona i nazwiska, gdy LLR znalazł proste powiązania jak "o których", "o którym".
Dla trigramów natomiast lepiej sprawdza się LLR, typ znajdowanych n-gramów się nie zmienił, lecz są one sensowniejsze. PMI zaczęło zawierać "lub", "dla", "na" na końcu trigramu, co nie dodaje mu wartości.
#### What types of expressions are discovered by the methods.
PMI - nazwy własne, imiona i nazwiska
LLR - często łączone części mowy: "o którym mowa", "o których mowa"

#### Can you devise a different type of filtering that would yield better results?
Zwiększyłbym dla w sumie dość dużego korpusu liczbę wystąpień, oraz zawierające krótkie słowa (<=2) bo są to zazwyczaj "się", "do", "o" co nam nie pomoże określić dziedziny tekstu, uzyskać sensownych n-gramów, które rzadko zawieraję tego typu słowa.